<h1 style="text-align:center">Data Science and Machine Learning Capstone Project</h1>
<img style="float:right" src="https://prod-edxapp.edx-cdn.org/static/edx.org/images/logo.790c9a5340cb.png">
<p style="text-align:center">IBM: DS0720EN</p>
<p style="text-align:center">Question 2 of 4</p>

1. [Problem Statement](#problem)
2. [Question 2](#question)
3. [Analyzing and Visualizing](#analysis)
4. [Concluding Remarks](#conclusion)

<a id="problem"></a>
## Problem Statement
---

The people of New York use the 311 system to report complaints about the non-emergency problems to local authorities. Various agencies in New York are assigned these problems. The Department of Housing Preservation and Development of New York City is the agency that processes 311 complaints that are related to housing and buildings.

In the last few years, the number of 311 complaints coming to the Department of Housing Preservation and Development has increased significantly. Although these complaints are not necessarily urgent, the large volume of complaints and the sudden increase is impacting the overall efficiency of operations of the agency.

Therefore, the Department of Housing Preservation and Development has approached your organization to help them manage the large volume of 311 complaints they are receiving every year.

The agency needs answers to several questions. The answers to those questions must be supported by data and analytics. These are their  questions:

<a id="question"></a>
## Question 2
---

Should the Department of Housing Preservation and Development of New York City focus on any particular set of boroughs, ZIP codes, or street (where the complaints are severe) for the specific type of complaints you identified in response to Question 1?

### Approach
Analyze the data to see if there is a higher correlation between the HEATING complaints and any particular borough, ZIP code, or street.

### Load Data
Separately the [New York 311](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9) data was loaded by [SODA](https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?$limit=100000000&Agency=HPD&$select=created_date,unique_key,complaint_type,incident_zip,incident_address,street_name,address_type,city,resolution_description,borough,latitude,longitude,closed_date,location_type,status) into a Pandas DataFrame then saved to a pickle file.

In [1]:
import pandas as pd
df = pd.read_pickle('C:\\Users\\It_Co\\Documents\\DataScience\\Capstone\\ny311full.pkl') # Local
#df = pd.read_pickle('./ny311.pkl') #IBM Cloud / Watson Studio
df.shape

(5862383, 15)

<a id="analysis"></a>
## Analyzing and Visualizing
---

### Reduce data to relevant rows and columns

In [2]:
#Remove rows that were not for the complaint types identified in question one.
df.drop(df[df["complaint_type"].isin(["HEAT/HOT WATER","HEATING"])==False].index, inplace=True)
#Double check that the correct rows were removed.
df['complaint_type'].value_counts()

HEAT/HOT WATER    1152592
HEATING            887869
Name: complaint_type, dtype: int64

In [3]:
#Remove columns deemed unnecessary for this question.
df.drop(['created_date','complaint_type','resolution_description','closed_date','location_type','status','address_type'], axis=1, inplace=True)
df.shape

(2040461, 8)

### Wrangle any unruly data

In [4]:
#Normalize strings so different casing won't appear as separate values.
df['incident_address'] = df['incident_address'].str.upper()
df['street_name'] = df['street_name'].str.upper()
df['city'] = df['city'].str.upper()
df['borough'] = df['borough'].str.upper()

In [5]:
#See if any data is null.
df.isnull().sum()

unique_key              0
incident_zip        18970
incident_address        1
street_name             1
city                18843
borough                 0
latitude            18966
longitude           18966
dtype: int64

<p style="color:Red;">How is it that the city could ever be missing, when the borough is not?</p>

In [6]:
df['borough'].value_counts()

BRONX            569960
BROOKLYN         543166
MANHATTAN        398552
UNSPECIFIED      282917
QUEENS           228447
STATEN ISLAND     17419
Name: borough, dtype: int64

<p style="color:Red;">The expected five boroughs, but then also:  UNSPECIFIED.  What does that mean?</p>

In [7]:
df[df["borough"]=="UNSPECIFIED"]["city"].value_counts()

BROOKLYN               93388
BRONX                  88585
NEW YORK               59095
JAMAICA                 5020
STATEN ISLAND           3462
ASTORIA                 3381
FLUSHING                3154
RIDGEWOOD               2273
FAR ROCKAWAY            2040
WOODSIDE                1773
ELMHURST                1696
JACKSON HEIGHTS         1479
CORONA                  1398
FOREST HILLS            1340
REGO PARK               1134
SOUTH RICHMOND HILL     1005
QUEENS VILLAGE           928
SUNNYSIDE                877
OZONE PARK               835
RICHMOND HILL            778
HOLLIS                   776
WOODHAVEN                770
EAST ELMHURST            752
SPRINGFIELD GARDENS      675
SAINT ALBANS             660
SOUTH OZONE PARK         606
KEW GARDENS              599
ARVERNE                  589
LONG ISLAND CITY         454
ROSEDALE                 416
OAKLAND GARDENS          397
MASPETH                  363
ROCKAWAY PARK            361
BAYSIDE                  236
COLLEGE POINT 

<p style="color:Red;"><b>Insight</b>:  When the borough is UNSPECIFIED it appears to mean that often either the borough <i>or even a "neighborhood" (a division below borough)</i> has been entered in the CITY column.  The city is actually "correct" with NEW YORK only 59K times.  The city column is a de-facto "neighborhood" column for the most part.</p>

<p style="color:Red;">Because borough is something further analysis will key from, fix the data where possible.</p>

In [8]:
import numpy as np
#Correct rows where borough was entered in the city column with "UNSPECIFIED" in the borough column.
five_boroughs = ["BROOKLYN","BRONX","MANHATTAN","QUEENS","STATEN ISLAND"]
which_rows_to_adjust = df[(df["borough"]=='UNSPECIFIED')&df["city"].isin(five_boroughs)].index
df.loc[which_rows_to_adjust,'borough']=df.loc[which_rows_to_adjust,'city']
df.loc[which_rows_to_adjust,'city']=np.nan

<p style="color:Red;">Almost 200K previously "UNSPECIFIED" rows will now show up under the correct borough during later analysis.</p>

In [9]:
#See if all the boroughs encompassed when the CITY is showing up as "New York"
df[df['city']=='NEW YORK']['borough'].value_counts()

MANHATTAN      393941
UNSPECIFIED     59095
Name: borough, dtype: int64

In [10]:
#See if all the MANHATTAN borough entries filled in the CITY as "New York" if if they sometimes have "neighborhood".
df[df['borough']=='MANHATTAN']['city'].value_counts()

NEW YORK    393941
BRONX            9
Name: city, dtype: int64

<p style="color:Red;"><b>Insight</b>:  Some data entered very oddly with BRONX as the city and MANHATTAN as the borough.</p>

<p style="color:Red;"><b>Decision</b>:  Ambiguous and only 9 rows out of millions, so drop the data so it won't confuse further analysis.</p>

In [11]:
print(df.shape) #before
#Drop a few rows of ambiguous data.
df.drop(df[(df["borough"]=='MANHATTAN')&(df["city"]=='BRONX')].index, axis=0, inplace=True)
print(df.shape) #after

(2040461, 8)
(2040452, 8)


<p style="color:Red;"><b>Decision</b>:  There are almost 400K rows with city as NEW YORK and the borough is MANHATTAN.  The other 59K rows with city NEW YORK has UNSPECIFIED as the borough.  Assume the borough is also MANHATTAN for those.</p>

In [12]:
#Fill in UNSPECIFIED borough when city was entered as NEW YORK.
which_rows_to_adjust = df[(df["borough"]=='UNSPECIFIED')&(df["city"]=='NEW YORK')].index
df.loc[which_rows_to_adjust,'borough']="MANHATTAN"
df.loc[which_rows_to_adjust,'city']=np.nan

In [13]:
#Although the city for most of the "NEW YORK" ones are the only ones that technically got the "city" column valued correctly,
#since every other row uses city as "neighborhood":  Make these consistent.
which_rows_to_adjust = df[(df["city"]=='NEW YORK')].index
df.loc[which_rows_to_adjust,'city']=np.nan

In [14]:
#Double check how many unspecified boroghs
df['borough'].value_counts()

BRONX            658545
BROOKLYN         636554
MANHATTAN        457638
QUEENS           228450
UNSPECIFIED       38384
STATEN ISLAND     20881
Name: borough, dtype: int64

<p style="color:Red;"><b>Decision</b>:  There are still 38K rows with unspecified boroughs, but the neighborhoods are in the city column, so use them to map to boroughs.</p>

In [15]:
#See the neighborhoods of the still unspecified boroughs.
neighborhoods = df[(df['borough']=='UNSPECIFIED')&(df['city'].isnull()==False)]['city'].unique()
neighborhoods.sort()
neighborhoods

array(['ARVERNE', 'ASTORIA', 'BAYSIDE', 'BELLEROSE', 'BREEZY POINT',
       'CAMBRIA HEIGHTS', 'COLLEGE POINT', 'CORONA', 'EAST ELMHURST',
       'ELMHURST', 'FAR ROCKAWAY', 'FLORAL PARK', 'FLUSHING',
       'FOREST HILLS', 'FRESH MEADOWS', 'GLEN OAKS', 'HOLLIS',
       'HOWARD BEACH', 'JACKSON HEIGHTS', 'JAMAICA', 'KEW GARDENS',
       'LITTLE NECK', 'LONG ISLAND CITY', 'MASPETH', 'MIDDLE VILLAGE',
       'NEW HYDE PARK', 'OAKLAND GARDENS', 'OZONE PARK', 'QUEENS VILLAGE',
       'REGO PARK', 'RICHMOND HILL', 'RIDGEWOOD', 'ROCKAWAY PARK',
       'ROSEDALE', 'SAINT ALBANS', 'SOUTH OZONE PARK',
       'SOUTH RICHMOND HILL', 'SPRINGFIELD GARDENS', 'SUNNYSIDE',
       'WHITESTONE', 'WOODHAVEN', 'WOODSIDE'], dtype=object)

<p style="color:Red;"><b>Insight</b>:  Checking https://en.wikipedia.org/wiki/List_of_Queens_neighborhoods show all but NEW HYDE PARK are in QUEENS.</p>

<p style="color:Red;"><b>Decision</b>:  NEW HYDE PARK isn't in any borough but is right on the border with QUEENS, and had only 2 complaints, so just consider it within QUEEENS.</p>

In [16]:
#Fix rows with unspecified borough but with a neighborhood (in the city column) that indicates it is in QUEEENS.
which_rows_to_adjust = df[(df["borough"]=='UNSPECIFIED')&df["city"].isin(neighborhoods)].index
df.loc[which_rows_to_adjust,'borough']="QUEENS"
df['borough'].value_counts()

BRONX            658545
BROOKLYN         636554
MANHATTAN        457638
QUEENS           266565
STATEN ISLAND     20881
UNSPECIFIED         269
Name: borough, dtype: int64

<p style="color:Red;">This will ultimately become the raw numbers to partially answer part of the the question.</p>

In [17]:
#Examine rows for those final 269 unspecified boroughs
df[df['borough']=='UNSPECIFIED'].describe()

,incident_zip,latitude,longitude
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


<p style="color:Red;"><b>Decision</b>:  Nothing "worth the trouble" to base further adjustments on for only 269 rows.  Set the borough to null on these rows so that they will not skew any subsequent analysis that examines borough.</p>

In [18]:
#Fix rows with unspecified borough and no other practical information from which to derive it.
which_rows_to_adjust = df[(df["borough"]=='UNSPECIFIED')&df["city"].isnull()].index
df.loc[which_rows_to_adjust,'borough']=np.nan

In [19]:
#Fix rows with city (neighborhood) equal to borough
which_rows_to_adjust = df[(df["borough"]==df["city"])].index
df.loc[which_rows_to_adjust,'city']=np.nan

In [20]:
#Fix rows with city that started off null or was adjusted subsequently to be null
which_rows_to_adjust = df[df["city"].isnull()].index
df.loc[which_rows_to_adjust,'city']="Unspecified"

<p style="color:Red;">At this point borough is reasonably populated with all the UNSPECIFIED and weird cases mitigated.</p>

<p style="color:Red;">The question calls for using zip code too.</p>

In [21]:
#There were some null zip codes seen before.  Examine them more closely.
df[df['incident_zip'].isnull()==True].describe(include="all")

,unique_key,incident_zip,incident_address,street_name,city,borough,latitude,longitude
count,18970,0.0,18969,18969,18970,18701,4.000000,4.000000
unique,18970,NaN,8306,2205,1,5,NaN,NaN
top,40931525,NaN,34 ARDEN STREET,GRAND CONCOURSE,Unspecified,BRONX,NaN,NaN
freq,1,NaN,232,365,18970,5957,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,40.724393,-73.849776
std,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
min,NaN,NaN,NaN,NaN,NaN,NaN,40.724393,-73.849776
25%,NaN,NaN,NaN,NaN,NaN,NaN,40.724393,-73.849776
50%,NaN,NaN,NaN,NaN,NaN,NaN,40.724393,-73.849776
75%,NaN,NaN,NaN,NaN,NaN,NaN,40.724393,-73.849776


<p style="color:Red;"><b>Decision</b>:  Nothing "worth the trouble" to base further adjustments for the missing zip codes.  If these 18K rows were deemed super important, a library for looking up a zip code based on an address would be necessary.  Just leave them as NaN values.</p>

In [22]:
#Learn the nature of the zip code data.
print(df[(df['incident_zip'].isnull()==True)].shape)
print(df[(df['incident_zip'].isnull()==False)].shape)
print(df[(df['incident_zip'].isnull()==False)&(df['incident_zip'] % 1.0 == 0.0)].shape)
print(df[(df['incident_zip']<10000.0)].shape)
print(df[(df['incident_zip']>99999.0)].shape)
print(df[(df['incident_zip']>9999.0)&(df['incident_zip']<100000.0)].shape)

(18970, 8)
(2021482, 8)
(2021482, 8)
(0, 8)
(0, 8)
(2021482, 8)


<p style="color:Red;"><b>Insight</b>:  The zip codes are all float64.  All of them have zero after the decimal point.  They are all five digits before the decimal point.  Almost 19K of the 2 Million+ rows are null.</p>

<p style="color:Red;">The question calls for using street too.</p>

In [23]:
#There was only one row with a null street.  Can it be repaired from address or anything?
df[df['street_name'].isnull()==True]

,unique_key,incident_zip,incident_address,street_name,city,borough,latitude,longitude
251660,21833329,NaN,NaN,NaN,Unspecified,BRONX,NaN,NaN


<p style="color:Red;"><b>Decision</b>:  Get rid of it.  Just one row.</p>

In [24]:
#Remove row with null street.
print(df.shape)
df.drop(df[df["street_name"].isnull()==True].index, inplace=True)
print(df.shape)

(2040452, 8)
(2040451, 8)


In [25]:
df['street_name'].value_counts()

GRAND CONCOURSE             35960
BROADWAY                    23497
OCEAN AVENUE                17882
ARDEN STREET                15841
MORRIS AVENUE               15792
ST NICHOLAS AVENUE          14945
AMSTERDAM AVENUE            11844
ELMHURST AVENUE             10946
BOYNTON AVENUE              10809
DR M L KING JR BOULEVARD    10037
OCEAN PARKWAY                9965
WALTON AVENUE                9579
BAILEY AVENUE                9561
RIVERSIDE DRIVE              9189
LINDEN BOULEVARD             9173
SEDGWICK AVENUE              9111
DECATUR AVENUE               9095
NOSTRAND AVENUE              8826
CRESTON AVENUE               8683
SHERMAN AVENUE               7885
BEDFORD AVENUE               7530
SHERIDAN AVENUE              7502
EASTERN PARKWAY              7424
3 AVENUE                     7323
WALLACE AVENUE               7287
2 AVENUE                     7172
DAVIDSON AVENUE              6840
FT WASHINGTON AVENUE         6698
NEW YORK AVENUE              6663
VALENTINE AVEN

<p style="color:Red;"><b>Insight</b>:  Before any attempt at normalization, there are 5979 distinct street_name values.</p>

# Go home and re-think your life.

In [134]:
import re

popular_suffixes = [
    "AVENUE","STREET","BOULEVARD","PLACE","ROAD","PARKWAY","CONCOURSE","DRIVE",
    "TERRACE","HIGHWAY","PARK","EXPRESSWAY","SQUARE","PLAZA","OVAL","CRESCENT",
    "LANE","COURT","EXTENSION","TURNPIKE", "LOOP","ESTATE"]
directionals = ["SOUTH","NORTH","EAST","WEST"]
popular_streets = ["BROADWAY","BOWERY"]

def street_mlk(street):
    effective_street = street
    effective_street = effective_street.replace("DR M L KING JR","mlk")
    effective_street = effective_street.replace("DR MARTIN L KING","mlk")
    effective_street = effective_street.replace("MARTIN LUTHER KING","mlk")
    effective_street = effective_street.replace("MARTIN L KING JR","mlk")
    effective_street = effective_street.replace("MARTIN L KING","mlk")
    effective_street = effective_street.replace("mlk","MARTIN LUTHER KING")
    return effective_street

def fix_last_word(street, old, new):
    split_street = street.split()
    last_index = len(split_street) - 1
    if split_street[last_index]==old:
        split_street[last_index] = new
    effective_street = " ".join(split_street)
    return effective_street

def fix_middle_words(street, old, new):
    split_street = street.split()
    found_at_index = next((i for i, x in enumerate(split_street) if x==old), None)
    last_index = len(split_street) - 1
    if found_at_index!=None:
        if found_at_index!=last_index:
            split_street[found_at_index] = new
    effective_street = " ".join(split_street)
    return effective_street

def normalize_street(street):
    effective_street = street.strip() #Remove leading and trailing spaces.
    effective_street = re.sub(' +', ' ', effective_street) #Squeeze multiple spaces to one space.
    
    effective_street = street_mlk(effective_street)
    effective_street = fix_last_word(effective_street,"AVE","AVENUE")
    effective_street = fix_last_word(effective_street,"ST","STREET")
    effective_street = fix_last_word(effective_street,"RD","ROAD")
    
    split_street = effective_street.split()
    last_word = split_street[-1]
   
    if street in popular_streets:
        return "accounted for - popular"
    if (len(split_street)==2)&(last_word in popular_suffixes):
        return "accounted for - 2 word normal"
    if (len(split_street)==2)&(split_street[0]=="AVENUE"):
        return "accounted for - avenue X"
    if (last_word in directionals)&(len(split_street)>2):
        if (split_street[-2] in popular_suffixes):
            return "accounted for - normal with directional"
    if (len(split_street)==2)&(split_street[0] in directionals):
        if (split_street[1] in popular_streets):
            return "accounted for - directional popular"
    
    return effective_street
    #return last_word

In [135]:
df['normalized_street_name'] = df['street_name'].apply(normalize_street)

In [136]:
df['normalized_street_name'].value_counts()

accounted for - 2 word normal              1403563
accounted for - normal with directional      28589
accounted for - popular                      23619
accounted for - avenue X                     18742
ST NICHOLAS AVENUE                           14945
MARTIN LUTHER KING BOULEVARD                 10244
FT WASHINGTON AVENUE                          6698
NEW YORK AVENUE                               6663
ADAM C POWELL BOULEVARD                       6427
EAST 21 STREET                                6013
ST JOHNS PLACE                                5861
EAST TREMONT AVENUE                           5809
EAST 19 STREET                                5659
ST MARKS AVENUE                               5292
WHITE PLAINS ROAD                             4907
FREDERICK DOUGLASS BOULEVARD                  4708
EAST 18 STREET                                4509
EAST 12 STREET                                3879
WEST 141 STREET                               3753
DE KALB AVENUE                 

In [81]:
df[df['normalized_street_name']=='SOUTH']['street_name'].value_counts()

EAST MOSHOLU PARKWAY SOUTH         2136
ANDREWS AVENUE SOUTH               1722
WEST MOSHOLU PARKWAY SOUTH         1558
BRONX PARK SOUTH                   1475
PELHAM PARKWAY SOUTH               1334
LORING PLACE SOUTH                  433
PARK LANE SOUTH                     280
SHORE PARKWAY SR SOUTH              256
PARK AVENUE SOUTH                   181
HOYT AVENUE SOUTH                    98
CENTRAL PARK SOUTH                   80
HOR HARDING EXPRESSWAY SR SOUTH      60
VAN CORTLANDT PARK SOUTH             56
CROTONA PARK SOUTH                   48
ASTORIA BOULEVARD SOUTH              31
DELANCEY ST SOUTH                    30
CROSS BRONX SERVICE ROAD SOUTH       25
7 AVENUE SOUTH                       24
NARROWS ROAD SOUTH                   12
GRAND CENTRAL PARKWAY SR SOUTH       11
VILLAGE ROAD SOUTH                    8
GRAND CENTRAL PARKWAY SOUTH           7
SUTTON PLACE SOUTH                    6
JUNIPER BLVD SOUTH                    4
229 DRIVE SOUTH                       3


In [ ]:
df[(df['street_name'].isnull()==False)&(df['street_name'].str.contains('  '))].head()

<p style="color:Red;"><b>Insight</b>:  The street names are not entirely normalized.  Some have multiple spaces in them.</p>

In [ ]:
df[df['street_name'].str.contains('CONCOURSE')]['street_name'].value_counts()

In [ ]:
#get  all words in the street_name column in a countable form.
words = list()
df['street_name'].str.split().apply(words.extend)
word_series = pd.Series(words)
counts = word_series.value_counts()
counts.shape

In [ ]:
counts.drop(counts[counts < 100].index, inplace=True) # Drop rare ones
counts.shape

In [ ]:
counts.drop(counts[counts.index.str.isdigit()].index, inplace=True) # Drop ones that are ONLY numbers
counts.shape

In [ ]:
counts['AVENUE']

In [ ]:
#AVENUE
#STREET
#BOULEVARD
#PLACE
#ROAD
#PARKWAY
#CONCOURSE
#PARK
#TERRACE


#EAST
#WEST
#SOUTH
#NORTH

<p style="color:Red;"><b>Finds</b>:  .</p>

### Visualize data

In [ ]:
graph_width_max = df['borough'].value_counts().max()

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt

In [ ]:
#Gather totals for graphic
totals_for_graph = df['borough'].value_counts()

In [ ]:
#Plot
totals_for_graph.plot(kind='barh', figsize=(13, 4), color='steelblue')
plt.xlim = (0, graph_width_max)
plt.xlabel('Number of complaints')
plt.title('HEATING / HOT WATER complaints by borough')
for index, value in enumerate(totals_for_graph): 
    label = format(int(value), ',') # format int with commas
    # place text at the end of bar (subtracting 47000 from x, and 0.1 from y to make it fit within the bar)
    plt.annotate(label, xy=(value - 47000, index - 0.10), color='white')
plt.show()

<a id="conclusion"></a>
## Concluding Remarks
---

xxx.

The Department of Housing Preservation and Development of New York City should focus on the following particular set of boroughs, ZIP codes, and streets (where the complaints are severe) for the "HEAT/HOT WATER" + "HEATING" complaint types:

<p style="color:Red;">xxx</p>

xxx